In [69]:
import openai
import os
import json

model="gpt-4o-mini"
openai.api_key = "sk-proj-2vPdQ1i59qOzikwXBQQAT3BlbkFJJtTiypSB67sWYpXQwmD6"

client = openai.OpenAI(api_key="sk-proj-2vPdQ1i59qOzikwXBQQAT3BlbkFJJtTiypSB67sWYpXQwmD6")

In [70]:
data_folder = "mt-metrics-eval-v2/wmt23/sources/"
out_folder = "new_translations/wmt23/system-outputs"
ref_suffix = ".refA.txt"
ref_folder = "mt-metrics-eval-v2/wmt23/references/"
mapping = {
    "cs": "Czech",
    "uk": "Ukrainian",
    "de": "German",
    "en": "English",
    "he": "Hebrew",
    "ru": "Russian",
    "zh": "Chinese",
}

# mt.json
# prompts_simulated.json
with open('mt_base.json', 'r') as file:
    prompts = json.load(file)

In [71]:
languages = ["de-en", "cs-uk", "en-zh"] #"de-en"
sample = 500

In [72]:
def load_sample(path, sample):
    loaded = []
    with open(path, 'r') as f:
        i = 0
        for line in f:
            if i >= sample:
                #i += 1
                #continue
                break
            loaded.append(line.strip())
            i += 1
    return loaded

In [75]:
import random
import re

def make_typos(sentence, prob_threshold=0.1, seed=42):
    new_sentence = ""
    i = 0
    random.seed(seed)
    while i < len(sentence):
        # Do not replace anything in placeholders - there is still the source sentence placeholder
        if sentence[i] == '[':
            close_i = sentence.find(']', i)
            new_sentence += sentence[i:close_i+1]
            i = close_i + 1
            continue
        # With a random probability of prob_threshold, introduce a typo
        if random.Random().random() < prob_threshold:
            # Only swap if not approaching the end of the sentence and if the characters are not special
            if i+1 < len(sentence) and not re.match(r'\W', sentence[i]) and not re.match(r'\W', sentence[i+1]):
                new_sentence += sentence[i+1] + sentence[i] #Swap two consecutive letters
                i += 2
                continue
            # Not used ATM, only swapping. Or omit the current letter (i.e don't copy it to the new string)
        else:
            new_sentence += sentence[i]
        i += 1
    return new_sentence

In [ ]:
import pandas as pd
for lp in languages:
    # load testset
    testset = load_sample(f'{data_folder}/{lp}.txt', sample)
    references = load_sample(f'{ref_folder}{lp}{ref_suffix}', sample)
    
    for prompt in prompts:
        for i in range(0, 110, 10):
            translations = []
            source_language = lp.split("-")[0]
            target_language = lp.split("-")[1]
            # Try the rest of the prompt variants
            #if "01-src" not in prompt['id']:
            #    continue
            template = prompt['prompt'].replace("[target language]", mapping[target_language]).replace("[source language]", mapping[source_language])
            experiment_name = f"{sample}_v2_" + prompt['id'] + f"_{i}"



            for item in testset:
                noised_template = make_typos(template, i/float(100), item)
                messages = [
                {"role": "system", "content": "You are a helpful machine translation assistant."},
                {"role": "user", "content": noised_template.replace('[source sentence]', item)}
        ]

                response = client.chat.completions.create(model=model, messages=messages)
                translation = response.choices[0].message.content
                translations.append(translation.replace('\n', ' ').replace('\t', ' '))

            # create folder translations/wmt23/system-outputs
            if not os.path.exists(f'new_translations/wmt23/system-outputs/{lp}'):
                os.makedirs(f'new_translations/wmt23/system-outputs/{lp}')
            dict = {'source': testset, 'translation': translations, 'reference': references}
            df = pd.DataFrame(dict)
            with open(f'new_translations/wmt23/system-outputs/{lp}/{experiment_name}.csv', 'w') as f:
                df.to_csv(f)
